In [2]:
import os
os.environ["PYTHONNET_RUNTIME"] = "coreclr"
os.environ["DOTNET_SYSTEM_DRAWING_USE_GDIPLUS"] = "1"

import clr
import numpy as np
import time
from datetime import timedelta
from pythonnet import load
from System import Environment, Array, String

# Load .NET Core runtime
load("coreclr")

# Try to import System.IO; fallback to os.chdir if it fails
try:
    from System.IO import Directory, Path, File
    use_dotnet_dir = True
except Exception as e:
    print(f"Note: System.IO import failed ({e}), using os.chdir instead")
    import os
    use_dotnet_dir = False

# Define DWSIM path
dwsimpath = "/usr/local/lib/dwsim/"

# Set working directory
if use_dotnet_dir:
    Directory.SetCurrentDirectory(dwsimpath)
else:
    os.chdir(dwsimpath)

# Add DWSIM assemblies
clr.AddReference(dwsimpath + "CapeOpen.dll")
clr.AddReference(dwsimpath + "DWSIM.Automation.dll")
clr.AddReference(dwsimpath + "DWSIM.Interfaces.dll")
clr.AddReference(dwsimpath + "DWSIM.GlobalSettings.dll")
clr.AddReference(dwsimpath + "DWSIM.SharedClasses.dll")
clr.AddReference(dwsimpath + "DWSIM.Thermodynamics.dll")
clr.AddReference(dwsimpath + "DWSIM.UnitOperations.dll")
clr.AddReference(dwsimpath + "DWSIM.Inspector.dll")
clr.AddReference(dwsimpath + "System.Buffers.dll")
clr.AddReference(dwsimpath + "DWSIM.Thermodynamics.ThermoC.dll")

# Now import DWSIM types
from DWSIM.Interfaces.Enums.GraphicObjects import ObjectType
from DWSIM.Thermodynamics import Streams, PropertyPackages
from DWSIM.UnitOperations import UnitOperations
from DWSIM.Automation import Automation3
from DWSIM.GlobalSettings import Settings

print("DWSIM imports successful!")

DWSIM imports successful!


In [3]:
# Create an instance of the Automation3 class from the DWSIM.Automation module
# This class provides methods for automating tasks in DWSIM, such as creating and manipulating flowsheets
interf = Automation3()

In [4]:
# Creates a flowsheet
sim = interf.CreateFlowsheet()

In [5]:
# Add Compounds
cnames = ["Water"]
sim.AddCompound("Water")

In [6]:
one = sim.AddFlowsheetObject('Material Stream','1')
two = sim.AddFlowsheetObject('Material Stream','2')
three = sim.AddFlowsheetObject('Material Stream','3')
four = sim.AddFlowsheetObject('Material Stream','4')
five = sim.AddFlowsheetObject('Material Stream','5')
six = sim.AddFlowsheetObject('Material Stream','6')
seven = sim.AddFlowsheetObject('Material Stream','7')
eight = sim.AddFlowsheetObject('Material Stream','8')
PUMP_1 = sim.AddFlowsheetObject('Pump','PUMP-1')
PUMP_2 = sim.AddFlowsheetObject('Pump','PUMP-2')
PUMP_3 = sim.AddFlowsheetObject('Pump','PUMP-3')
PUMP_4 = sim.AddFlowsheetObject('Pump','PUMP-4')
E_1 = sim.AddFlowsheetObject('Energy Stream','E1')
E_2 = sim.AddFlowsheetObject('Energy Stream','E2')
E_3 = sim.AddFlowsheetObject('Energy Stream','E3')
E_4 = sim.AddFlowsheetObject('Energy Stream','E4')

In [7]:
one = one.GetAsObject()
two = two.GetAsObject()
three = three.GetAsObject()
four = four.GetAsObject()
five = five.GetAsObject()
six = six.GetAsObject()
seven = seven.GetAsObject()
eight = eight.GetAsObject()
PUMP_1 = PUMP_1.GetAsObject()
PUMP_2 = PUMP_2.GetAsObject()
PUMP_3 = PUMP_3.GetAsObject()
PUMP_4 = PUMP_4.GetAsObject()
E_1 = E_1.GetAsObject()
E_2 = E_2.GetAsObject()
E_3 = E_3.GetAsObject()
E_4 = E_4.GetAsObject()

In [8]:
sim.ConnectObjects(one.GraphicObject , PUMP_1.GraphicObject, -1, -1)
sim.ConnectObjects(E_1.GraphicObject , PUMP_1.GraphicObject, -1, -1)
sim.ConnectObjects(PUMP_1.GraphicObject , two.GraphicObject, -1, -1)
sim.ConnectObjects(three.GraphicObject , PUMP_2.GraphicObject, -1, -1)
sim.ConnectObjects(E_2.GraphicObject , PUMP_2.GraphicObject, -1, -1)
sim.ConnectObjects(PUMP_2.GraphicObject , four.GraphicObject, -1, -1)
sim.ConnectObjects(five.GraphicObject , PUMP_3.GraphicObject, -1, -1)
sim.ConnectObjects(E_3.GraphicObject , PUMP_3.GraphicObject, -1, -1)
sim.ConnectObjects(PUMP_3.GraphicObject , six.GraphicObject, -1, -1)
sim.ConnectObjects(seven.GraphicObject , PUMP_4.GraphicObject, -1, -1)
sim.ConnectObjects(E_4.GraphicObject , PUMP_4.GraphicObject, -1, -1)
sim.ConnectObjects(PUMP_4.GraphicObject , eight.GraphicObject, -1, -1)

In [9]:
sim.AutoLayout()

In [10]:
one.SetOverallComposition(Array[float]([1]))
one.SetTemperature(350.0) # K
one.SetPressure(101325.0) # Pa
one.SetMassFlow(1.0) # kg/s

three.SetOverallComposition(Array[float]([1]))
three.SetTemperature(350.0) # K
three.SetPressure(101325.0) # Pa
three.SetMassFlow(1.0) # kg/s

five.SetOverallComposition(Array[float]([1]))
five.SetTemperature(350.0) # K
five.SetPressure(101325.0) # Pa
five.SetMassFlow(1.0) # kg/s

seven.SetOverallComposition(Array[float]([1]))
seven.SetTemperature(350.0) # K
seven.SetPressure(101325.0) # Pa
seven.SetMassFlow(1.0) # kg/s

E_4.SetEnergyFlow(2) # kW

In [11]:
# property package
Steam_table = sim.CreateAndAddPropertyPackage("Steam Tables (IAPWS-IF97)")

In [12]:
# Calc Modes
PUMP_1.CalcMode = PUMP_1.CalcMode.Delta_P
PUMP_1.DeltaP = 201325
PUMP_1.Eficiencia = 72

PUMP_2.CalcMode = PUMP_2.CalcMode.OutletPressure
PUMP_2.Pout = 201325
PUMP_2.Eficiencia = 76

PUMP_3.CalcMode = PUMP_3.CalcMode.Power
PUMP_3.set_DeltaQ(2)

PUMP_4.CalcMode = PUMP_4.CalcMode.EnergyStream

In [13]:
# request a calculation
errors = interf.CalculateFlowsheet4(sim)
list(errors)

[]

In [15]:
# save file

fileNameToSave = Path.Combine(Environment.GetFolderPath(Environment.SpecialFolder.Desktop), "/workspace/00 FlowSheet Automation/03 Pump Automation/03 Pump.dwxmz")

interf.SaveFlowsheet(sim, fileNameToSave, True)